In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# coefficients
h = 0.020 # timestep in seconds

R_o = 0.0958
R_i = 0.0438
R_b = 0.0077
I_b = 1.28e-6
m_b = 0.032
b   = 1.3e-6
g   = 9.81

# from https://github.com/aa4cc/flying-ball-in-hoop/blob/master/m/params_init.m
ath1 = -0.4800
ath2 = -5.1195
ath3 = 0.0677
bth  = 586.3695

apsi1 = -0.0034
apsi2 = -73.6336
apsi3 = -0.3351
bpsi  = 210.12

a_bar = I_b * (R_o ** 2)/(R_b ** 2) + m_b * (R_o - R_b) ** 2
b_bar = b * (R_o / R_b) ** 2
c_bar = 0.0298
d_bar = -b_bar
e_bar = I_b * (R_o / R_b) * (R_o / (R_b + 1))

In [ ]:
# state is a vector that contains th(theta), Dth(derivative of theta), 
# psi, Dpsi, r, Dr, phi, Dphi, and mode as specified in the paper
state = {
    'th': 0.,
    'Dth' : 0.,
    'psi' : 0.,
    'Dpsi' : 0.,
    'r' : 0.,
    'Dr' : 0.,
    'phi' : 0.,
    'Dphi' : 0.,
    'mode' : 1,
}

In [ ]:
def update_outerhoop(state, t_i)
    '''
    Updates the state as it rolls on the outer hoop
    '''
    assert state['mode'] == 1
    
    th_dot = state['Dth']
    Dth_dot = ath1*state['Dth'] + ath2*sin(state['psi']) + ath3*state['Dpsi'] + bth*t_i
    psi_dot = state['Dpsi']
    Dpsi_dot = apsi1*state['Dth'] + apsi2*sin(state['psi']) + apsi3*state['Dpsi'] + bth*t_i
    
    state['th'] += h * th_dot
    state['Dth'] += h * Dth_dot
    state['psi'] += h * Dpsi
    state['Dpsi'] += h * Dpsi_dot    
    state['r'] = state['r']
    state['Dr'] = 0.
    state['phi'] = (state['th'] - state['psi'])*R_o/R_b
    state['Dphi'] = (state['Dth'] - state['Dpsi'])*R_o/R_b
    state['mode'] = 1
    
    return state

In [ ]:
def outer_to_free(state, eps=0.):
    '''
    Determines whether to transition from the outer hoop to free fall
    and if so, returns the modified state
    '''
    assert state['mode'] == 1
        
    if -g*cos(state['psi'])-((R_o - R_b)*(state['dpsi']**2)) > eps:
        
        state['phi'] = (state['th'] - state['psi']) * R_o/R_b
        state['Dphi'] = ((R_o + R_b)/R_b) * state['Dth'] - (R_o/R_b)* state['Dpsi']
        state['r'] = R_o - R_b
        state['Dr'] = 0
        state['mode'] = 2
        
        return state
    
    return state

In [ ]:
def update_freefall(state, t_i):
    '''
    Update the state as it is in freefall
    '''
    assert state['mode'] == 2
    
    th_dot = state['Dth']
    Dth_dot = ath1*state['Dth'] + bth*t_i
    phi_dot = state['Dphi']
    Dphi_dot = 0
    r_dot = Dr
    Dr_dot = state['r'] * state['Dpsi']**2 + g * cos(state['psi'])
    psi_dot = state['Dpsi']
    Dpsi_dot = -(g * sin(state['psi']) + 2*state['Dpsi']*state['Dr'])/state['r']
    
    state['th'] += h * th_dot
    state['Dth'] += h * Dth_dot
    state['psi'] += h * psi_dot
    state['Dpsi'] += h * Dpsi_dot
    state['r'] += h * r_dot
    state['Dr'] += h * Dr_dot
    state['phi'] += h * phi_dot
    state['Dphi'] += h * Dphi_dot
    state['mode'] = 2
    
    return state

In [ ]:
def free_to_outer(state, eps=0):
    '''
    Determines whether to transition from free fall to the outer hoop
    and changes the state if it does
    '''
    assert state['mode'] == 2
    
    if state['r'] - R_o + R_b > eps:
        
        state['Dpsi'] = state['Dth'] - (R_b / R_o) * state['Dphi'] + state['Dpsi']
        state['Dr'] = 0.
        state['r'] = R_o - R_b
        state['mode'] = 1
        
        return state
    
    return state

In [ ]:
def free_to_inner(state, eps=0):
    '''
    Determines whether to transition from free fall to the inner hoop
    and changes the state if it does
    '''
    assert state['mode'] == 2
    
    if R_i + R_b - state['r'] > eps:
        
        state['Dpsi'] = state['Dth'] + (R_b/R_i) * state['Dphi'] + state['Dpsi']
        state['Dr'] = 0.
        state['r'] = R_i + R_b
        state['mode'] = 3
        
        return state
    
    return state

In [ ]:
def update_inner(state, t_i):
    '''
    Update the state as the ball is on the inner hoop
    '''
    assert state['mode'] == 3
    
    th_dot = state['Dth']
    Dth_dot = ath1*state['Dth'] + bth*t_i
    psi_dot = state['Dpsi']
    Dpsi_dot = (1/a_bar) * (-b_bar * state['Dpsi'] - c_bar*sin(state['psi']) - d_bar * state['Dth'] + e_bar*t_i)
    
    state['th'] += h * th_dot
    state['Dth'] += h * Dth_dot
    state['psi'] += h * psi_dot
    state['Dpsi'] += h * Dpsi_dot
    state['r'] = state['r']
    state['Dr'] = 0.
    state['phi'] = -(state['th'] - state['psi']) * (R_i / R_b)
    state['Dphi'] = -(state['Dth'] - state['Dpsi']) * (R_i / R_b)
    state['mode'] = 3
    
    return state    

In [ ]:
def inner_to_free(state, eps=0):
    '''
    Determines whether to transition from inner hoop
    to free fall and changes the state if it does
    '''
    assert state['mode'] == 3
    
    if g * cos(state['psi']) + (R_i + R_b) * state['Dpsi']**2 > eps:
        
        state['phi'] = -(state['th'] - state['psi']) * (R_i / R_b)
        state['Dphi'] = -(((R_i - R_b) / R_b) * state['Dth'] - (R_i / R_b) * state['Dpsi'])
        state['r'] = R_i + R_b
        state['Dr'] = 0.
        state['mode'] = 2
        
        return state
    
    return state